In [1]:
%%capture
#@title
import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientdet" not in os.getcwd():
  !git clone --depth 1 https://github.com/rezabasiri/EfficientDetDFU
  os.chdir('EfficientDetDFU/automl/efficientdet')
  sys.path.append('.')
  !pip install -r requirements.txt
  !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
else:
  !git pull

In [2]:
ckpt = '20200715-efficientdet-d1-finetune'
MODEL = 'efficientdet-d1'

def download(m):
  if m not in os.listdir():
    !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=1ES-0mYRQVTLbAC4SuDWOJCprZ9P7Enpu" > /tmp/intermezzo.html
    !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > FINAL_DOWNLOADED_FILENAME.zip
    !unzip FINAL_DOWNLOADED_FILENAME.zip
    !rm -rf FINAL_DOWNLOADED_FILENAME.zip
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(ckpt)
print('Use ckpt in {}'.format(ckpt_path))

# Alternatively access the checkpoint from https://drive.google.com/file/d/11L4jYRgCBUzchWt35SOMc2v_dmgZdxat/view?usp=sharing

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3274    0  3274    0     0  12125      0 --:--:-- --:--:-- --:--:-- 12125
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1632      0 --:--:-- --:--:-- --:--:--  1632
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 97.6M    0 97.6M    0     0  38.5M      0 --:--:--  0:00:02 --:--:-- 71.1M
Archive:  FINAL_DOWNLOADED_FILENAME.zip
   creating: 20200715-efficientdet-d1-finetune/
  inflating: 20200715-efficientdet-d1-finetune/model.ckpt-13474.data-00000-of-00001  
  inflating: 20200715-efficientdet-d1-finetune/model.ckpt-19359.data-00000-of-00001  
  inflating: 20200715-efficie

In [3]:
if MODEL not in os.listdir():
  !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{MODEL}.tar.gz
  !tar zxf {MODEL}.tar.gz

--2020-07-25 23:14:53--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d1.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 108.177.125.128, 64.233.187.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49059930 (47M) [application/octet-stream]
Saving to: ‘efficientdet-d1.tar.gz’

efficientdet-d1.tar 100%[===================>]  46.79M  21.4MB/s    in 2.2s    

2020-07-25 23:14:56 (21.4 MB/s) - ‘efficientdet-d1.tar.gz’ saved [49059930/49059930]



## Prepare data

In [4]:
min_score_thresh = 0.10
max_boxes_to_draw = 2
line_thickness = 2

# first export a saved model.
saved_model_dir = 'savedmodel'
!rm -rf {saved_model_dir}
!python model_inspect.py --runmode=saved_model --model_name={MODEL} \
  --ckpt_path={ckpt_path} --saved_model_dir={saved_model_dir} \
  --hparams="num_classes=2,max_instances_per_image =4,first_lr_drop_epoch=60,second_lr_drop_epoch=160,learning_rate=0.0002,moving_average_decay=0,mixed_precision=true,autoaugment_policy=v3" \

2020-07-25 23:15:03.172952: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-25 23:15:05.689720: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2300000000 Hz
2020-07-25 23:15:05.690195: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2967100 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-07-25 23:15:05.690235: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-07-25 23:15:05.695938: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-25 23:15:05.904342: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-25 23:15:05.905518: I tensorflow/compiler/xla/servic

In [ ]:
serve_image_out = 'serve_image_out'
!mkdir {serve_image_out}

!python model_inspect.py --runmode=saved_model_infer \
  --saved_model_dir={saved_model_dir} \
  --model_name={MODEL}  --input_image="{Path_To_Test_Folder}/*.jpg"  \
  --output_image_dir={serve_image_out} \
  --min_score_thresh={min_score_thresh}  --max_boxes_to_draw={max_boxes_to_draw} \

In [ ]:
from google.colab import drive
drive.mount('/content/drive')